In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler

chat_model = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)


In [23]:
from langchain.prompts import ChatPromptTemplate\

make_poem_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an excellent poet. You must write a concise and sophisticated poem about a given programming language. The answer format should consist of the title of the poem and the content of the poem. Please add a blank line at the end."),
        ("human", "{language}"),
    ]
)

pome_exp_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a literary critic with excellent linguistic skills. You should describe in detail a poem written on the topic of programming languages."),
        ("human", "{poem}"),
    ]
)

make_poem_chain = make_poem_template | chat_model 

poem_exp_chain = pome_exp_template | chat_model

final_chain = {"poem" : make_poem_chain} | poem_exp_chain

final_chain.invoke(
    {
        "language": "javascript"
    }
)

Title: The Dance of JavaScript

In the realm of web and code,
Where languages meet and nodes,
There dances one with grace untold,
JavaScript, bright and bold.

With functions, arrays, and loops,
It weaves through pages, jumps and swoops,
Interacting with the user's touch,
Bringing websites alive as such.

From front-end to back-end it spans,
A versatile language in many hands,
Dynamic, flexible, and free,
JavaScript, the heart of web's grand spree.

So let us raise a toast today,
To the language that lights our way,
In the world of bits and bytes so vast,
JavaScript, forever will last."The Dance of JavaScript" is a captivating poem that elegantly captures the essence of the programming language JavaScript within the context of web development. The title itself sets the stage for a metaphorical interpretation of JavaScript as a dancer, moving gracefully and boldly through the realm of code and web.

The poem opens with a vivid imagery of the coding world as a realm where languages conve

AIMessageChunk(content='"The Dance of JavaScript" is a captivating poem that elegantly captures the essence of the programming language JavaScript within the context of web development. The title itself sets the stage for a metaphorical interpretation of JavaScript as a dancer, moving gracefully and boldly through the realm of code and web.\n\nThe poem opens with a vivid imagery of the coding world as a realm where languages converge and nodes connect, setting the scene for JavaScript to take center stage. The use of rhyme and rhythm throughout the poem adds a musical quality to the verses, enhancing the overall flow and engaging the reader in the dance of words.\n\nThe poet skillfully personifies JavaScript as a dancer with "grace untold," emphasizing its dynamic and versatile nature. The language is depicted as weaving through pages, interacting with users, and bringing websites to life with its functions, arrays, and loops. This portrayal not only highlights the technical aspects of